In [4]:
import pandas as pd
import torch
import os

In [ ]:
train_df = pd.read_csv("C:/Users/Manasa/OneDrive/Desktop/Drug_Repurposing_Gnn/Drug_Repurposing_Gnn/data/train_inductive.csv")
val_df   = pd.read_csv("C:/Users/Manasa/OneDrive/Desktop/Drug_Repurposing_Gnn/Drug_Repurposing_Gnn/data/val_inductive.csv")
test_df  = pd.read_csv("C:/Users/Manasa/OneDrive/Desktop/Drug_Repurposing_Gnn/Drug_Repurposing_Gnn/data/test_inductive.csv")


TypeError: read_csv() got an unexpected keyword argument 'index'

In [11]:
df = pd.read_csv("C:/Users/Manasa/OneDrive/Desktop/Drug_Repurposing_Gnn/Drug_Repurposing_Gnn/data/processed/triples.csv", sep=",", header=None,
                 names=["head", "relation", "tail"])


C:\Users\Manasa\AppData\Local\Temp\ipykernel_16552\778138026.py:1: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/Manasa/OneDrive/Desktop/Drug_Repurposing_Gnn/Drug_Repurposing_Gnn/data/processed/triples.csv", sep=",", header=None,


In [9]:
entities = pd.read_csv("C:/Users/Manasa/OneDrive/Desktop/Drug_Repurposing_Gnn/Drug_Repurposing_Gnn/data/processed/entities.csv")
entity_types = {}
for ent in entities["entity"]:
    if ent.startswith("DrugBank"):
        entity_types[ent] = "Drug"
    elif ent.startswith("Gene") or ent.startswith("Protein"):
        entity_types[ent] = "Gene"
    elif ent.startswith("Disease") or ent.startswith("DOID"):
        entity_types[ent] = "Disease"
    else:
        entity_types[ent] = "Other"



In [12]:
keep_entities = {e for e, t in entity_types.items() if t in ["Drug", "Gene", "Disease"]}
df = df[df["head"].isin(keep_entities) & df["tail"].isin(keep_entities)]

# Keep only relevant relations
keywords = ["treat", "target", "associate", "indication", "palliate"]
mask = df["relation"].str.contains("|".join(keywords), case=False, regex=True)
df = df[mask]

print(f"Remaining triples: {len(df)}")

Remaining triples: 0


In [13]:
# Step 4: Build vocabularies

entities = sorted(set(df["head"]).union(set(df["tail"])))
relations = sorted(set(df["relation"]))
ent2id = {e: i for i, e in enumerate(entities)}
rel2id = {r: i for i, r in enumerate(relations)}

In [14]:
# Step 5: Build edge_index

src = [ent2id[h] for h in df["head"]]
dst = [ent2id[t] for t in df["tail"]]
etype = [rel2id[r] for r in df["relation"]]

edge_index = torch.tensor([src, dst], dtype=torch.long)
edge_type = torch.tensor(etype, dtype=torch.long)

print("Graph ready:", edge_index.shape, edge_type.shape)


Graph ready: torch.Size([2, 0]) torch.Size([0])


In [15]:
# Step 6: Temporal split

cutoff_year = 2017
train_df = df[df["year"] <= cutoff_year]
test_df  = df[df["year"] > cutoff_year]

# Validation = 10% from train
val_df = train_df.sample(frac=0.1, random_state=42)
train_df = train_df.drop(val_df.index)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

KeyError: 'year'